### Asynchronous SGD updates using Distributed Cache Redis

1. Create spot instances
2. Mount EFS (Not required for our project)
3. Start redis server in all the instances and create a cluster
4. Pull the code from github repo (https://github.com/SrujithPoondla/vanilla-hogwild.git)
5. If need to divide the dataset between the nodes run the specific cell
6. Run the scripts to start training
7. After training ends close the instances

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from aws_setup import *
from argparse import ArgumentParser
import boto3
sys.argv = ['foo']

#### Define parameters

In [3]:
parser = ArgumentParser(description='Asynchronous SGD updates using Redis')
parser.add_argument('--n-nodes', type=int, default=1, metavar='N',
                    help='how many aws instances to start')
parser.add_argument('--is-redis', action='store_true', default = True,
                    help="Choose whether the model to be trained using redis or not."
                    "If not using Redis, model will be trained on single process")


_StoreTrueAction(option_strings=['--is-redis'], dest='is_redis', nargs=0, const=True, default=True, type=None, choices=None, help='Choose whether the model to be trained using redis or not.If not using Redis, model will be trained on single process', metavar=None)

In [4]:
args = parser.parse_args()
vpc_name='vpc-1b056b60'
args.n_nodes = 3
n_instances = args.n_nodes
instance_type = 'm5.2xlarge'
ami_sr = 'ami-e9ce4896'
a_zone = 'us-east-1a'

if args.n_nodes is 2 and args.is_redis:
    print('Cant create a cluster with 2 redis nodes. Chose either a 3 node cluster or single instance')

#### Get Existing VPC by tag name

In [5]:
vpc = get_vpc(vpc_name); vpc

ec2.Vpc(id='vpc-1b056b60')

#### Create EFS (if you haven't already)

In [6]:
# efs_tag = f'{vpc_name}-efs'

In [7]:
# efs = create_efs(efs_tag, vpc, performance_mode='maxIO')

#### Request Spot instance

In [8]:
instance_name = f'{vpc_name}-instance'
# Recommend a high compute instance as we need to do multi-threaded resizing later on

In [9]:
spot_price = get_spot_prices()[instance_type]
bid_price = "%.4f" % (float(spot_price)*3)
print(f'Spot price: {spot_price}, Bid price: {bid_price}')

Spot price: 0.128600, Bid price: 0.3858


In [10]:
launch_specs = LaunchSpecs(vpc, instance_type=instance_type, ami= ami_sr,availability_zone=a_zone).build()

In [11]:
# launch_specs['BlockDeviceMappings'][0]['Ebs']['VolumeSize'] = 1000

In [12]:
launch_specs

{'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
   'Ebs': {'DeleteOnTermination': True,
    'VolumeSize': 20,
    'VolumeType': 'gp2'}}],
 'ImageId': 'ami-e9ce4896',
 'InstanceType': 'm5.2xlarge',
 'KeyName': 'aws-key-spot-instance',
 'NetworkInterfaces': [{'AssociatePublicIpAddress': True,
   'DeviceIndex': 0,
   'Groups': ['sg-2624da6f'],
   'SubnetId': 'subnet-10d9d04d'}]}

In [13]:
ec2 = boto3.resource('ec2')
filters = [
    {
        'Name': 'instance-state-name',
        'Values': ['running']
    }
]
# filter the instances based on filters() above
ec2_instances = list(ec2.instances.filter(Filters=filters))
instances = []
for instance in ec2_instances:
    instances.append(instance)
instances_to_request = n_instances-len(instances)
instances = create_multiple_spot_instance(instance_name, launch_specs,instance_count=args.n_nodes, spot_price=bid_price)
print(instances)

Keypair exists
Waiting on spot fullfillment...
Fulfillment completed. InstanceId: i-0940a02f36a361c86
Waiting on spot fullfillment...
Fulfillment completed. InstanceId: i-0ca8348dc6cc1b8ce
Waiting on spot fullfillment...
Fulfillment completed. InstanceId: i-0f026bc05da4366cf
Rebooting...
Completed. SSH:  ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@34.207.132.124
Completed. SSH:  ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@35.173.252.14
Completed. SSH:  ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@54.210.248.242
[ec2.Instance(id='i-0940a02f36a361c86'), ec2.Instance(id='i-0ca8348dc6cc1b8ce'), ec2.Instance(id='i-0f026bc05da4366cf')]


In [14]:
# instance = get_instance(instance_name); instance
ssh_commands = []
public_ip_list = []
private_ip_list = []
for instance in instances:
    # for each instance, append to lists
    private_ip_list.append(instance.private_ip_address)
    public_ip_list.append(instance.public_ip_address)
    ssh_commands.append(get_ssh_command(instance))
print(ssh_commands, public_ip_list, private_ip_list)

['ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@34.207.132.124', 'ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@35.173.252.14', 'ssh -i ~/.ssh/aws-key-spot-instance.pem ubuntu@54.210.248.242'] ['34.207.132.124', '35.173.252.14', '54.210.248.242'] ['10.0.0.14', '10.0.0.11', '10.0.0.10']


#### Terminating instances

In [16]:
# for instance in instances:
#     print(instance.terminate())

{'TerminatingInstances': [{'CurrentState': {'Code': 32, 'Name': 'shutting-down'}, 'InstanceId': 'i-045cd7cb9c5349d14', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': '5bc7cae3-41f7-47b4-a0a7-57005563ff67', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8', 'transfer-encoding': 'chunked', 'vary': 'Accept-Encoding', 'date': 'Sat, 05 May 2018 09:37:02 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}
{'TerminatingInstances': [{'CurrentState': {'Code': 32, 'Name': 'shutting-down'}, 'InstanceId': 'i-0842f1c4c49a737c2', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': 'c2b5f4af-a13d-4fb1-8bd3-bd22964652b0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8', 'transfer-encoding': 'chunked', 'vary': 'Accept-Encoding', 'date': 'Sat, 05 May 2018 09:37:02 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}
{'TerminatingInstances': [{'CurrentState': {'Code': 32, 'N

### SSH

In [24]:
clients = []
for instance in instances:
    clients.append(connect_to_instance(instance))
print(clients)

Connecting to SSH...
Got client
/Users/srujithpoondla/.ssh/aws-key-spot-instance.pem
Connected!
Connecting to SSH...
Got client
/Users/srujithpoondla/.ssh/aws-key-spot-instance.pem
Connected!
Connecting to SSH...
Got client
/Users/srujithpoondla/.ssh/aws-key-spot-instance.pem
Connected!
[<paramiko.client.SSHClient object at 0x115ac8898>, <paramiko.client.SSHClient object at 0x115ac8e48>, <paramiko.client.SSHClient object at 0x115b77908>]


#### Mount EFS

In [25]:
# efs_addr = get_efs_address('fast-ai-efs'); efs_addr

In [26]:
# _ = run_command(client, 'mkdir ~/efs_mount')

In [27]:
# efs_mount_cmd = f'sudo mount -t nfs -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 {efs_addr}:/ ~/efs_mount'
# _ = run_command(client, efs_mount_cmd)

In [28]:
# _ = run_command(client, 'ls efs_mount') # no reformatting

## Tmux

In [40]:
tsess = []
for client in clients:
    if 'sess' not in run_command(client,'tmux ls'):
        tsess.append(TmuxSession(client, 'sess'))         
print(tsess)

[<aws_setup.TmuxSession object at 0x115b870f0>, <aws_setup.TmuxSession object at 0x115d86da0>, <aws_setup.TmuxSession object at 0x115db4e48>]


#### Activate Conda Environment in all the instances and check whether we need to create a cluster or not. Then start redis using conf files.

In [30]:
for client,sess in zip(clients,tsess):
    if 'redis' not in run_command(client,'tmux ls'):
        redis = TmuxSession(client, 'redis-sess')
    run_command(client, 'cd ~/vanilla-hogwild && git stash && git pull && git checkout stable')
    if (args.n_nodes >= 3):
        print(redis.run_command('~/miniconda3/envs/largescale/bin/redis-server ~/redis-conf/redis_cluster.conf'))
    else:
        print(redis.run_command('~/miniconda3/envs/largescale/bin/redis-server ~/redis-conf/redis.conf &&'))
  

('', '')
('', '')
('', '')


#### Create redis cluster

In [31]:
if args.n_nodes >= 3:
    ip_str = ''
    for ip in private_ip_list:
        ip_str = ip_str+ ip +":6379 "
    print(ip_str)
    redis = TmuxSession(clients[0],'redis-serv-sess')
    redis.run_command('cd /home/ubuntu/redis-4.0.9/src && ./redis-trib.rb create '+ ip_str)
    redis.run_command('yes')


10.0.0.14:6379 10.0.0.11:6379 10.0.0.10:6379 


#### Creating Arguments String

In [33]:
#Model parameters
batch_size = 128
epochs = 20
lr = 0.01
momentum = 0.5
log_interval = 50
num_processes = 2
nnet_arch = 'LeNet'
dataset = 'cifar10'
args.is_redis = True

In [51]:
hosts = ''
for ip in private_ip_list:
    hosts = hosts+ip+','
hosts = hosts.strip(',')

10.0.0.14,10.0.0.11,10.0.0.10


In [52]:
#dataset can be 'MNIST' or 'cifar10'
#architecture can be 'LeNet' or 'ResNet'(still working on this)
#num_processes should be either 1 or 2
#batch size 128,256,512,1024,2048

arg_str = '--is-redis='+str(args.is_redis)+' --dataset='+dataset+' --nnet-arch='+nnet_arch+' --num-processes='+\
str(num_processes) + ' --batch-size='+str(batch_size) +' --lr='+str(lr) + ' --hosts='+hosts

print(arg_str)

--is-redis=True --dataset=cifar10 --nnet-arch=LeNet --num-processes=2 --batch-size=128 --lr=0.01 --hosts=10.0.0.14,10.0.0.11,10.0.0.10


#### Chose the log file name

In [45]:
if args.is_redis:
    log_file = dataset+'-'+nnet_arch+'-'+str(batch_size)+'-'+str(num_processes)+'-'+ 'redis'
else:
    log_file = dataset+'-'+nnet_arch+'-'+str(batch_size)+'-'+str(num_processes)
print('Log file name: '+log_file)

Log file name: cifar10-LeNet-128-2-redis


In [46]:
out = run_command(clients[0],'w')[0].split(',')[1].strip().split(' ')[0]

In [47]:
count = 0
running = True
for sess in tsess:
    sess.run_command('source activate largescale')
    sess.run_command('python3 -u ~/vanilla-hogwild/main.py '+arg_str+ '2>&1 | tee '+log_file)
#     sess.run_command('exit')
while(running):
    for client in clients:
        if int(run_command(clients[0],'w')[0].split(',')[1].strip().split(' ')[0]) > 4:
            continue
        else:
            count = count+1
        if count==len(clients):
            running = False
            break

for instance in instances:
    print(instance.terminate())
for ip in public_ip_list:
    ec2c.release_address(PublicIp=ip)

KeyboardInterrupt: 

In [ ]:
crea